In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

import data_collection

In [2]:
dfs = data_collection.etl()

(0/17) https://seatgeek.com/nets-at-76ers-tickets/1-25-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775973
(1/17) https://seatgeek.com/nuggets-at-76ers-tickets/1-28-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775975
(2/17) https://seatgeek.com/magic-at-76ers-tickets/1-30-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775977
(3/17) https://seatgeek.com/magic-at-76ers-tickets/2-1-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775979
(4/17) https://seatgeek.com/knicks-at-76ers-tickets/2-10-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775981
(5/17) https://seatgeek.com/rockets-at-76ers-tickets/2-13-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775983
(6/17) https://seatgeek.com/cavaliers-at-76ers-tickets/2-15-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775985
(7/17) https://seatgeek.com/grizzlies-at-76ers-tickets/2-23-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775987
(8/17) https://seatgeek.com/celtics-at-76ers-

In [3]:
master_df = pd.concat(dfs)
master_df['section'] = master_df['section'].apply(lambda x: x.split(',')[0])

section_size = master_df.groupby('section').size()
valid_sections = section_size.index.values[section_size > 3]
valid_sections

array(['Club Box 13', 'Club Box 14', 'Club Box 2', 'Club Box 24',
       'Section 101', 'Section 102', 'Section 103', 'Section 104',
       'Section 105', 'Section 106', 'Section 107', 'Section 108',
       'Section 109', 'Section 110', 'Section 111', 'Section 112',
       'Section 113', 'Section 114', 'Section 115', 'Section 116',
       'Section 118', 'Section 119', 'Section 120', 'Section 122',
       'Section 123', 'Section 124', 'Section 203', 'Section 204',
       'Section 205 A', 'Section 206', 'Section 207', 'Section 208',
       'Section 209', 'Section 209 A', 'Section 210', 'Section 210 A',
       'Section 211', 'Section 212', 'Section 214', 'Section 215',
       'Section 220', 'Section 221', 'Section 221 A', 'Section 222',
       'Section 222 A', 'Section 223', 'Section 224', 'Section R 106'],
      dtype=object)